In [1]:
import numpy as np
import matplotlib.pyplot as plt
import heapq

In [2]:
 def game_simulation(nb_p = 4, rounds=10, delta=10**-5):   
    """ input:
        - nb_p      : number of players (3 or 4)
        - rounds    : number of rounds per player's A choice to account for random optimal choices
        - delta     : minimal distance between two players choices

    notes:
    - for each player "i" :
        -  i_options   : distances of segments that player "i" can chose to cover 
        -  i_positions : positions that player "i" can choose from, directly associated with i_options
        -  i_idx       : indices of i_options that are optimal in given circumstances 
        -  i_choice    : the final choice that player i makes 

    - other arrays used:
        - dist         : array that updates lenghts of segments seperated by player's choices in the range [0,1] 
        - dist_coverage: array that calculates finals distance covered by each player
        - cum_coverage : array collecting total distances covered by each player in all rounds of the simulation
        - ns           : utility array used for segregation of player's choices """ 
    # =========================================================================================================================                                 
    # discretization of player's A choices
    A_range = np.linspace(0, 0.5, 1001)               
    dist_coverage = np.zeros(nb_p)                  
    cum_coverage = np.zeros((len(A_range),nb_p))  
    ns = np.zeros(nb_p) 
    # =========================================================================================================================
    # iterating over player's A possible choices
    for i, A in enumerate(A_range):                 
        for _ in range(rounds):          
            # dist array initiated based on player's A choice           
            dist = np.array([A, 1-A])
            # =================================================================================================================
            # Player B
            B_options = np.array([dist[0]-delta, 1/nb_p*(dist[1])])                      
            B_positions = np.array([B_options[0],1-B_options[1]])                         
            B_idx  = np.asarray(np.where(np.isclose(B_options, max(B_options)).flatten())).flatten() 
            # if more than one optimal choice, random choice is made 
            B_choice = np.random.choice(B_idx) if len(B_idx) >=2 else B_idx                             
            
            if B_choice == 0:
                B = B_positions[0]
                dist = np.array([B, dist[0]-B, dist[1]])
            else:
                B = B_positions[1]
                dist = np.array([dist[0], B-dist[0], (1-B)])
        
            # =================================================================================================================
            # Player C 
            C_options = np.array([dist[0]-delta, dist[1]/2, dist[2]-delta])
            C_positions = np.array([dist[0]-delta, dist[0]+dist[1]/2, dist[0]+dist[1]+delta])
            C_idx  = np.asarray(np.where(np.isclose(C_options, max(C_options)).flatten())).flatten() 
            C_choice = np.random.choice(C_idx) if len(C_idx) >=2 else C_idx 
            
            if C_choice == 0:
                C = C_positions[0]
                dist = np.array([C, dist[0]-C, dist[1], dist[2]])
            elif C_choice == 1:
                C = C_positions[1]
                dist = np.array([dist[0], C-dist[0], np.sum(dist[0:2])-C, dist[2]])
            else:
                C = C_positions[2]
                dist = np.array([dist[0], dist[1],C-np.sum(dist[0:2]), 1-C])

            # =================================================================================================================
            # Player D - if 4 players game
            if nb_p == 4:
                D_options = np.array([dist[0]-delta, dist[1]/2, dist[2]/2, dist[3]-delta])
                D_positions = np.array([dist[0]-delta, dist[0]+dist[1]/2, np.sum(dist[0:2])+dist[2]/2, np.sum(dist[0:3])+delta])
                D_idx  = np.asarray(np.where(np.isclose(D_options, max(D_options)).flatten())).flatten() 
                D_choice = np.random.choice(D_idx) if len(D_idx) >= 2 else D_idx 

                if D_choice == 0:
                    D = D_positions[0]
                    dist = np.array([D, dist[0]-D, dist[1], dist[2], dist[3]])
                elif D_choice == 1:
                    D = D_positions[1]
                    dist = np.array([dist[0], C-dist[0], np.sum(dist[0:2])-D, dist[2], dist[3]])
                elif D_choice == 2:
                    D = D_positions[2]
                    dist = np.array([dist[0], dist[1], D-np.sum(dist[0:2]), np.sum(dist[0:3])-D, dist[3]])
                else:
                    D = D_positions[3]
                    dist = np.array([dist[0], dist[1], dist[2], D-np.sum(dist[0:3]), 1-D])

            # =================================================================================================================
            # Final data manipulations
            players  = np.array([A, B, C, D]) if nb_p==4 else np.array([A, B, C])
            ns = np.array([heapq.nsmallest(nb_p, players)[k] for k in range(nb_p)])

            # determining distances covered by each player
            dist_coverage[np.where(ns[0]==players)] = ns[0] + (ns[1]-ns[0])/2
            dist_coverage[np.where(ns[1]==players)] = (ns[1]-ns[0])/2 + (ns[2]-ns[1])/2
            
            if nb_p==4:
                dist_coverage[np.where(ns[2]==players)] = (ns[2]-ns[1])/2 + (ns[3]-ns[2])/2
                dist_coverage[np.where(ns[3]==players)] = (ns[3]-ns[2])/2 + 1-ns[3]
            elif nb_p==3:
                dist_coverage[np.where(ns[2]==players)] = (ns[2]-ns[1])/2 + 1-ns[2]

            # collecting distances covered by each player per round
            cum_coverage[i,:] += dist_coverage/rounds
            # =================================================================================================================
    return A_range, cum_coverage, nb_p

# 3 players game simulation
A_range, cum_coverage, nb_p = game_simulation(nb_p = 3, rounds = 10, delta = 10**-5)
print(f"In {nb_p} players game, player's A best choice is: {A_range[np.argmax(cum_coverage[:,0])]}.")

# 4 players game simulation
A_range, cum_coverage, nb_p = game_simulation(nb_p = 4, rounds = 10, delta = 10**-5)
print(f"In {nb_p} players game, player's A best choice is: {A_range[np.argmax(cum_coverage[:,0])]}.")

In 3 players game, player's A best choice is: 0.25.
In 4 players game, player's A best choice is: 0.1995.
